# HR Dataset - Statistics Review

## Explore the data

The data set we will use for this exercise comes from a Kaggle challenge and is often used for predictive analytics, namely to predict why the best and most experienced employees tend to leave the company.  We won't be using it for any predictive purposes here, but will instead use this data set to review many of the concepts explored in the Statistical Inference lectures.

This data contains fields for various measures of employee performance and reported satisfaction levels, as well as categorical variables for events and salary level.  For now, just explore the data a bit to get a general idea of what is going on.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

% matplotlib inline

In [3]:
# Load data into a dataframe
data = pd.read_csv('HR_comma_sep.csv')

In [10]:
# Simple exploration
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_project           14999 non-null int64
average_montly_hours     14999 non-null int64
time_spend_company       14999 non-null int64
Work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
sales                    14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
satisfaction_level,14999.0,0.612834,0.248631,0.09,0.44,0.64,0.82,1.0
last_evaluation,14999.0,0.716102,0.171169,0.36,0.56,0.72,0.87,1.0
number_project,14999.0,3.803054,1.232592,2.00,3.00,4.00,5.00,7.0
average_montly_hours,14999.0,201.050337,49.943099,96.00,156.00,200.00,245.00,310.0
time_spend_company,14999.0,3.498233,1.460136,2.00,3.00,3.00,4.00,10.0
Work_accident,14999.0,0.144610,0.351719,0.00,0.00,0.00,0.00,1.0
left,14999.0,0.238083,0.425924,0.00,0.00,0.00,0.00,1.0
promotion_last_5years,14999.0,0.021268,0.144281,0.00,0.00,0.00,0.00,1.0


In [7]:
data.describe(include=['O']).T

,count,unique,top,freq
sales,14999,10,sales,4140
salary,14999,3,low,7316


In [11]:
# Rename two columns to fix typo, convert to lowercase
data.rename(columns={'average_montly_hours': 'average_monthly_hours',
                     'Work_accident': 'work_accident'}, inplace=True)
data.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Probability, Expectation Values, and Variance

The concepts of probability, expectation values, and variance are the bedrock of statistical inference.  Let's begin by employing some of these concepts to see if we can find some interesting paths to go down which may provide some insight into the inner workings of this company.

1. What is the probability that a randomly selected employee left the company?  What about experienced a work accident?  Also compute the probability that a randomly selected employee left the company and experienced a work accident.
1. Compute the 25th, 50th, and 90th percentiles for the satisfaction level score for all employees that left the company.  Compare these results to the same percentiles for those that did not leave.  What can you say about the results?
1. Compute the variance and standard deviation of hours worked.
1. Compare the variance between the satisfaction levels of employees who left versus those who stayed.  Which is larger?  What does this mean?
1. Compute the mean satisfaction level for each salary category.  Comment on your results.
1. Given an employees salary level (low, medium, or high), calculate the probability that they worked more than two standard deviations of the average monthly hours across all groups.  In other words, compute
$$P(hours > 2\sigma \vert salary ) = \dfrac{P(salary \vert hours > 2\sigma) P(hours > 2\sigma)}{P(salary)}$$
1. What can you say about your results in part 6?
1. Repeat parts 6 and 7 for 
$$P(left \vert salary ) = \dfrac{P(salary \vert left) P(left)}{P(salary)}$$
1. What is the odds ratio of an employee with a high salary getting a promotion within the past five years versus a low salary employee?  Comment on your results.
1. Suppose we were to pull a random sample of size 50 of employee satisfaction levels.  What would approximately be the mean of this sample?  What would be the mean of, say, 10 sets of random samples?  Demonstrate your assertions by writing some python code to do just that.


In [26]:
# Question 1
# Get total number of employees that left, find proportion of total
left_count = data.left.value_counts()[1]
prob_left = left_count / len(data.left)

# Get total number of employees who experienced an accident, find proportion of total
accident_count = data.work_accident.value_counts()[1]
prob_accident = accident_count / len(data.work_accident)

# Get total number of employees who left and had an accident, find proportion of total
left_accident_count = len(data[(data.left == 1) & (data.work_accident == 1)])
prob_left_accident = left_accident_count / len(data.left)

print('Probability of selecting an employee that left: %0.2f%%' % (prob_left*100))
print('Probability of selecting an employee that had an accident: %0.2f%%' % (prob_accident*100))
print('Probability of selecting an employee that left and had an accident: %0.2f%%' % (prob_left_accident*100))

Probability of selecting an employee that left: 23.81%
Probability of selecting an employee that had an accident: 14.46%
Probability of selecting an employee that left and had an accident: 1.13%


In [30]:
# Question 2
# Get dataframes for those who stayed and left
left_df = data[data.left == 1]
stay_df = data[data.left == 0]

l1, l2, l3 = left_df.satisfaction_level.quantile(q=[0.25, 0.5, 0.9])
s1, s2, s3 = stay_df.satisfaction_level.quantile(q=[0.25, 0.5, 0.9])

print('Satisfaction levels for 25th, 50th, and 90th percentile for employees who left:')
print('{}, {}, {}'.format(l1, l2, l3))
print('Satisfaction levels for 25th, 50th, and 90th percentile for employees who stayed:')
print('{}, {}, {}'.format(s1, s2, s3))


Satisfaction levels for 25th, 50th, and 90th percentile for employees who left:
0.13, 0.41, 0.84
Satisfaction levels for 25th, 50th, and 90th percentile for employees who stayed:
0.54, 0.69, 0.94


In [35]:
# Alternative solution using groupby
data.groupby('left').satisfaction_level.quantile(q=[0.25, 0.5, 0.9]).unstack()

,0.25,0.5,0.9
left,,,
0,0.54,0.69,0.94
1,0.13,0.41,0.84


In [45]:
# Question 3
var_hours_worked = data['average_monthly_hours'].var()
std_hours_worked = data['average_monthly_hours'].std()
mean_hours_worked = data['average_monthly_hours'].mean()

print('Variance of hours worked: {0:.2f}'.format(var_hours_worked))
print('Standard deviation of hours worked: {0:.2f}'.format(std_hours_worked))

Variance of hours worked: 2494.31
Standard deviation of hours worked: 49.94


In [36]:
# Alternative
data.average_monthly_hours.agg(['var', 'std'])

var    2494.313175
std      49.943099
Name: average_monthly_hours, dtype: float64

In [44]:
# Question 4
data.groupby('left').satisfaction_level.var()

left
0    0.047134
1    0.069661
Name: satisfaction_level, dtype: float64

In [43]:
# Question 5
data.groupby('salary').satisfaction_level.mean()

salary
high      0.637470
low       0.600753
medium    0.621817
Name: satisfaction_level, dtype: float64

In [47]:
# Question 6
ave_hours_plus2std = mean_hours_worked + (2 * std_hours_worked)

# Compute the components of Bayes theorem
N = len(data)

# P(hours > ave hours plus2)
p_hours = len(data[data.average_monthly_hours > ave_hours_plus2std]) / N

# P(salary), just proportions of each salary type in data
p_salary = data.salary.value_counts() / N

# P(salary | hours > ave hours plus2)
# The number of each salary type that works > ave hours plus2
# divided by the total number of workers who work > ave hours plus2
p_salary_hours = (data[data.average_monthly_hours > ave_hours_plus2std].salary.value_counts() /
                     len(data[data.average_monthly_hours > ave_hours_plus2std]))

# Finally use Bayes Theorem
p_hours_salary = (p_salary_hours * p_hours) / p_salary

In [48]:
p_hours_salary

low       0.013532
medium    0.008998
high      0.001617
Name: salary, dtype: float64

In [49]:
# Question 7
p_hours

0.01060070671378092

In [50]:
p_salary

low       0.487766
medium    0.429762
high      0.082472
Name: salary, dtype: float64

In [52]:
p_salary_hours

low       0.622642
medium    0.364780
high      0.012579
Name: salary, dtype: float64

## Distributions and The Central Limit Theorem
### The Bernoulli Distribution
Bernoulli distributions are the result of a random variable with a binary outcome, like a coin flip or medical test giving a positive or negative result.  Typically we represent the outcomes of a Bernoulli Random variable $X$ of only taking values of 0 or 1, with probabilities $p$ and $1 - p$ respectively, mean $p$, variance $p(1 - p)$, and PMF given by

$$ P(X = x) = p^x (1 - p)^{1 - x} $$

Where $x$ is the outcome and $p$ is the probability of the positive outcome (1).

Bernoulli random variables crop up very often in statistical analysis &mdash; most often in the form of Binomial trials, or, as a sum of independent Bernoulli variables with PMF given by 
$$ P(X = x) = {n \choose x} p^x (1 - p)^{n - x} $$
where
$$ {n \choose x} = \frac{n!}{x!(n - x)!} $$
In this exercise you'll take a look at the HR data and apply these concepts to gain some insight.

Using the HR data, answer the following.
1. Which variables in the HR data can be said to be Bernoulli random variables?
2. For the variables you identified in part 1, compute the probabilities $p_k$, of each having a positive $(x = 1)$ result, where $k$ is a placeholder for each variable.
3. Compute the variance of each of the variables in part 2 using $p_k$ as described above.
4. For each of the k variables, compute the probability of randomly selecting 3500 employees with a positive result.  Comment on your answer.
5. For each of the k variables, compute the probability of randomly selecting 3500 **or less** with a positive result.  Comment on your answer.
6. Now plot both the PMF and CDF as a function of the number of drawn samples for each of the k variables.  Comment on your results.

### The Normal Distribution
The Normal distribution (or sometimes called the Bell Curve or Gaussian) is by far the most prevalent and useful distribution in any field that utilizes statistical techniques.  In fact, in can be shown that the means of random variables sampled repeatedly from **any** distribution eventually form a normal given a sufficiently large sample size.

A normal distribution is characterized by the PDF given by
$$p(x|\mu,\sigma) = \frac{1}{\sqrt{(2\pi\sigma^2)}}e^{-\frac{(x - \mu)^2}{2\sigma^2}} $$

where $\mu$ is the mean and $\sigma^2$ is the variance, thus the distribution is characterized by mean and variance alone.  In this exercise, you'll examine some of the variables in the HR dataset and construct some normal distributions approximating them.

Using the HR data, answer the following

1. Which variables may be approximately normal?
2. For the variables in part 1, plot some histograms.
3. Compute the mean and variance for each of the variables used in parts 1 and 2.
4. Using the mean and variance in part 3, construct normal distributions for each and overlay them on top of the histograms you made in part one.  Are they well approximated by normals?

### The Poisson Distribution
The Poisson distribution is very versatile but is typically used to model counts per unit time or space, such as the number of ad clicks or arriving flights, each per unit time. It has a PDF given by
$$ P(X = x, \lambda) = \frac{\lambda^x e^{-\lambda}}{x!} $$
where the mean and variance are both equal to $\lambda$

Using the HR data, answer the following.

1. What variables would be good candidates for modeling with a Poisson distribution?
2. For each variable in part 1, divide each by salary and fit a Poisson distribution to each.
3. For each salary level, compute the probability of obtaining at least the mean of each variable &mdash; regardless of salary level &mdash; by using the Poisson distributions you constructed in part 2.  Comment on your results.

### The Central Limit Theorem
The Central Limit Theorem is perhaps one of the most remarkable results in statistics and mathematics in general.  In short, it says that the distribution of means of independent random variables, sampled from **any** distribution, tends to approach a normal distribution as the sample size increases.

An example of this would be taking a pair of dice, rolling them, and recording the mean of each result.  The Central Limit Theorem states, that after enough rolls, the distribution of the means will be approximately normal.  Stated formally, the result is
    $$ \bar{X_n} \sim N(\mu, \sigma^2/n) = \frac{\sqrt{n}}{\sigma \sqrt{2\pi}}e^{-n(\bar{X_n} - \mu)^2/\sigma^2}$$
In this exercise, you'll conduct some simulation experiments to explore this idea.

Using the HR data, answer the following.
1. Choose two variables which may be good candidates to test this theorem.
2. Using the variables chosen in part 1, randomly select a set of `n = 10` samples and take the mean.  Repeat this 1000 times for each variable.
3. Plot a histogram for each variable used in part 2.  Comment on your results.
4. Repeat parts 2-3 for `n = 100`, `n = 500`, and `n = 1000`.  Comment on your results.
5. Overlay an normal curve on your `n = 1000` plots, using the mean and variance computed from the data.  Comment on your results.

## Hypothesis Testing
Hypothesis testing is essentially using the data to answer questions of interest.  For example, does a new medication provide any benefit over placebo?  Or is a subset of the population disproportionately more susceptible to a particular disease?  Or is the difference between two companies profits' significant or due to chance alone?

Before doing some hypothesis testing on the HR data, recall that hypothesis typically come in pairs of the form $H_0$, called the null hypothesis, versus $H_a$, called the alternative hypothesis.  The null hypothesis represents the "default" assumption -- that a medication has no effect for example, while the alternative hypothesis represents what exactly we are looking to discover, in the medication case, whether it provides a significant benefit.  Another common case is testing the difference between two means.  Here, the null hypothesis is that there is no difference between two population means, whereas the alternative hypothesis is that there is a difference.  Stated more precisely
$$H_0: \mu_1 - \mu_2 = 0$$
$$H_a: \mu_1 - \mu_2 \ne 0$$

Hypotheses are usually tested by constructing a confidence interval around the test statistic and selecting a "cut-off" significance level denoted $\alpha$.  A typical $\alpha$ significance is 0.05 and is often called a "p-value".  If a test produces a p-value of $\alpha$ or below, then the null hypothesis can be rejected, strengthening the case of the alternative hypothesis.  It is very important to remember that hypothesis testing can only tell you if your hypothesis is statistically significant -- this does **not** mean that your result may be scientifically significant which requires much more evidence.

In this exercise you'll explore the HR data more and test some hypothesis.

Using the HR data, answer the following.

1. Compute a confidence interval for satisfaction levels, at the 95% confidence level, of employees who left the company and those who didn't.  Do this using both a t distribution and a normal.  Comment on your results.
2. Use a t-test to test the hypothesis that employees who left the company, had lower satisfaction levels than those who did not.  If significant, is the mean difference?  Comment on your results.  (Hint: Do the two populations have equal variance?)
3. Fit a normal curve to each group in part 2 and put them on the same plot next to each other.  Comment on your results.
4. Test the hypothesis that the satisfaction level between each salary group, denoted k, differs signicantly from the mean.  Namely
    - $H_0: \mu - \mu_k = 0$
    - $H_a: \mu - \mu_k \ne 0$
5. How would you interpret your results in part 5?
6. Generate plots for part 5 as you did in part 3.  What conclusions can you draw from the plot?
7. Repeat parts 4-6 on a hypothesis of your choosing.
8. Recall that Power is the probability of failing to reject the null hypothesis when it is false (thus more power is good).  Compute the power for the hypothesis that the satisfaction level of high paid employees is different than that of medium paid employees using a t distribution.

## Bootstrapping
Bootstrapping is an immensely useful technique in practice.  Very often you may find yourself in a situation where you want to compute some statistic, but lack sufficient data to do so.  Bootstrapping works as a remedy to this problem.

Recall that the bootstrapping algorithm breaks down as follows:
1. Sample n observations with replacement from the observed data resulting in one simulated complete data set. 
1. Take the statistic of the simulated data set
1. Repeat these two steps B times, resulting in B simulated statistics
1. These statistics are approximately drawn from the sampling distribution of the statistic of n observations
    - This is a lot like what you did when drawing many sample means

In this exercise you will implement this algorithm on the HR data.

Write a function that can perform bootrapping for the median of a set of n samples in the HR data set.  Test this function on the `satisfaction_level` with `n = 100` and `b = 100` and compare your results to the true median.  Also compute the standard deviation of the bootstrapped median.